In [28]:
import pylab
from nt_toolbox.graph import *
from nt_toolbox.signal import *
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import os
from PIL import Image

In [29]:
def binary(image, alpha=0.5):
    image_copy = np.zeros(image.shape)
    image_copy[image > alpha] = 1
    return image_copy

In [30]:
def has_points_in_images(img):
    '''
    img : image 
    
    return : True or False
             True : there are points inside the cell 
             False : there aren't points inside the cell
    '''
    img = img*img
    max_value = np.max(img)
    min_value = np.min(img)
    mean_value = np.mean(img)
    b_img1 = binary(img, mean_value)
    b_img2 = binary(img, (min_value + mean_value)/2)
    return abs(np.sum(b_img1-b_img2)) != 0

In [51]:
def extract(images_paths, label):
    p_detected = []
    var_color = []
    max_color = []
    var_gray = []
    max_gray = []
    labels = []
    
    for ip in images_paths:
        image = load_image(ip)
        #print(image[15])
        image_grey = np.asarray(Image.open(ip).convert('L'))
        p_detected.append(1 if has_points_in_images(image) else 0)
        var_color.append(np.array(image)[:,0].var())
        max_color.append(np.array(image)[:,0].max())
        var_gray.append(image_grey.var())
        max_gray.append(image_grey.max())
        labels.append(label)
        
    df = pd.DataFrame()
    #df['image_path'] = images_paths
    df['has_points'] = p_detected
    df['var_color'] = var_color
    df['max_color'] = max_color
    df['var_gray'] = var_gray
    df['max_gray'] = max_gray
    df['label'] = labels
    
    return df

## Test

In [61]:
for dirname, _, filenames in os.walk('../cell_images/Uninfected'):
    paths = []
    for filename in filenames[:500]:
        image_path = os.path.join(dirname, filename)
        paths.append(image_path)
    df1 = extract(paths, 0)
    '''
    for filename in filenames[:100]:
        image_path = os.path.join(dirname, filename)
        if image_path.endswith('.db'):
            continue
        image = load_image(image_path)
        p_labels.append(True)
        if not has_points_in_images():
            p_detected.append(True)
        else:
            imageplot(image)
            p_detected.append(False)
            print(image_path)    
    
    '''
    
for dirname, _, filenames in os.walk('../cell_images/Parasitized'):
    paths = []
    for filename in filenames[:500]:
        image_path = os.path.join(dirname, filename)
        paths.append(image_path)
    df2 = extract(paths, 1)

df = df1.append(df2)

# shuffle data
df = df.sample(frac=1)
print(df)

# train = 60% / test = 20% / validation = 20%
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

print(train)

features = ["has_points", "var_color", "max_color", "var_gray", "max_gray"]

train.to_csv("malaria_train.data", sep=' ', columns = features, index=False)
train.to_csv("malaria_train.solution", sep=' ', columns = ["label"], index=False)

test.to_csv("malaria_test.data", sep=' ', columns = features, index=False)
test.to_csv("malaria_test.solution", sep=' ', columns = ["label"], index=False)

validate.to_csv("malaria_valid.data", sep=' ', columns = features, index=False)
validate.to_csv("malaria_valid.solution", sep=' ', columns = ["label"], index=False)

     has_points  var_color  max_color     var_gray  max_gray  label
352           0   0.116352   0.973214  5097.844169       184      0
415           0   0.078136   0.958974  5687.109392       198      0
134           0   0.143426   0.983079  4659.484924       193      0
413           0   0.117359   0.970588  7476.516679       193      0
92            1   0.083525   0.940984  3947.780981       203      1
272           0   0.093819   0.968912  7671.905004       193      0
335           1   0.056328   0.977548  6415.493043       195      1
22            1   0.186812   0.921739  5154.553792       194      1
213           1   0.005836   0.978339  5634.624139       188      1
377           0   0.090694   0.930605  6645.380450       188      0
479           1   0.114245   0.892617  5045.095230       200      1
234           0   0.151085   0.900000  4491.879624       191      0
116           0   0.005541   0.970537  6962.834441       194      0
27            0   0.105487   0.981878  5397.2145

In [50]:
sum(df['has_points'] ==[0 for i in range(len(df['has_points']))] ) / len(df['has_points'])

0.409